# Build Structure and Calculate Bispectrum Coefficients
<b> nanoHUB tools by: </b>  <i>Mackinzie S. Farnell, Zachary D. McClure</i> and <i>Alejandro Strachan</i>, Materials Engineering, Purdue University <br>

In this notebook, we build an FCC structure with Cr, Fe, Co, Ni, and/or Cu and determine the bispectrum coefficients and nearest neighbors for each atom in the structure. We can use this information, along with a model trained on equiatomic CrFeCoNi, to predict properties for the structure.

Outline
1. Define Inputs
2. Run LAMMPS
3. Assign Nearest Neighbors


In [ ]:
# import libraries we will need
%load_ext yamlmagic
import numpy as np
import sys
from simtool import DB
from scipy.optimize import curve_fit
import random
import os
import subprocess
import json
import pandas as pd

## 1. Define Inputs
We define the input parameters for the LAMMPS simulation. LAMMPS (Large-scale Atomic/Molecular Massively Parallel Simulator) is a molecular dynamics simulator that we will use to build the structure and calculate the bispectrum coefficients. The LAMMPS simulation has several inputs including: 
- composition of the structure (our default is equiatomic CrFeCoNi)
- desired crystal structure (FCC)
- lattice parameter (3.56)

In [ ]:
# input parameters 
material = ['Co','Cr','Cu','Fe','Ni']
atom_number = {'Co': 27, 'Cr': 24, 'Cu':  29, 'Fe':26,'Ni':28}
crystal_structure = 'fcc'
lattice_parameter = 3.56
mass = {'Co': 58.933,'Cr': 51.996,  'Cu': 63.546, 'Fe': 55.845, 'Ni': 58.6934}

# sets composition of structure, compositions must add to 1
composition_Co = 0.25
composition_Cr = 0.25
composition_Cu = 0.0
composition_Fe = 0.25
composition_Ni = 1 - (composition_Co + composition_Cr + composition_Cu + composition_Fe)
composition_sum = (composition_Co + composition_Cr + composition_Cu + composition_Fe + composition_Ni)
  
composition_array = np.array([composition_Co, composition_Cr, composition_Cu, composition_Fe, composition_Ni]) 

rand_seed = 37
box_length = 10

# cutoff to determine nearest neighbors
cutoff = np.sqrt(2)/2 * lattice_parameter
model_name = 'CoCrCuFeNi.set' #input which potential to use

print("Composition\nPercent_Co: % .2f\nPercent_Cr: % .2f\nPercent_Cu: % .2f\nPercent_Fe: % .2f\nPercent_Ni: % .2f"
    % (composition_array[0], composition_array[1], composition_array[2], composition_array[3], composition_array[4]))

### Specify Number of Atoms
We specify that 4000 atoms should be in the structure and determine how many of each atom type we want based on the desired composition.

In [ ]:
num_atoms = 4*(box_length**3)

# determines the number of atoms per type
natoms_per_type = np.array([np.floor(composition_Co*num_atoms), np.floor(composition_Cr*num_atoms),
                            np.floor(composition_Cu*num_atoms), np.floor(composition_Fe*num_atoms),
                            np.floor(composition_Ni*num_atoms)], dtype = 'int')

# sets the number of Ni atoms so that number of atoms adds up correctly
natoms_per_type[-1] = num_atoms - np.sum(natoms_per_type[0:-1])

# sets true composition of structure based on number of atoms
true_comp_Co = natoms_per_type[0] / num_atoms
true_comp_Cr = natoms_per_type[1] / num_atoms
true_comp_Cu = natoms_per_type[2] / num_atoms
true_comp_Fe = natoms_per_type[3] / num_atoms
true_compNi = natoms_per_type[4] / num_atoms

print("Total Number of Atoms: % d\nNumber Co: % d\nNumber Cr % d\nNumber Cu % d\nNumber Fe % d\nNumber Ni % d\n"
   % (num_atoms, natoms_per_type[0], natoms_per_type[1], natoms_per_type[2], natoms_per_type[3], natoms_per_type[4]))

### Assign Atom IDs
We randomly assign each atom a unique identification number ranging from one to the total number of atoms in the structure. The ID numbers are stored in atoms.txt and are used to refer to atoms in subsequent code.

In [ ]:
np.random.seed(rand_seed)

# base list starts with all the atom ids (1-num_atoms)
base_list = list(range(1, num_atoms+1))

# randomly select ids from base list to be each atom
atom_Co = list(np.random.choice(base_list, natoms_per_type[0], replace=False))
base_list = list(set(base_list) - set(atom_Co))

atom_Cr = list(np.random.choice(base_list, natoms_per_type[1], replace=False))
base_list = list(set(base_list) - set(atom_Cr))

atom_Cu = list(np.random.choice(base_list, natoms_per_type[2], replace=False))
base_list = list(set(base_list) - set(atom_Cu))

atom_Fe = list(np.random.choice(base_list, natoms_per_type[3], replace=False))
base_list = list(set(base_list) - set(atom_Fe))

atom_Ni = list(np.random.choice(base_list, natoms_per_type[4], replace=False))
base_list = list(set(base_list) - set(atom_Ni))

# create a file called atom_types.txt to store all the atom ids
f = open('atom_types.txt','w')
for i in atom_Co:
    f.write('set atom %i type 1\n' % i)
for i in atom_Cr:
    f.write('set atom %i type 2\n' % i)
for i in atom_Cu:
    f.write('set atom %i type 3\n' % i)
for i in atom_Fe:
    f.write('set atom %i type 4\n' % i)
for i in atom_Ni:
    f.write('set atom %i type 5\n' % i)
f.close()

### Create Dictionary of Inputs
We place all inputs in a dictionary for eash access. 

In [ ]:
# put inputs in a dict for easy substitution with python format
inputs = {
    'material': material,
    'crystal_structure': crystal_structure, 
    'lattice_parameter': lattice_parameter,
    'mass': mass,
    'atom_number':atom_number.values(),
    'model_name': model_name,
    'box_length': box_length,
    'cutoff':cutoff,
}

## 2. Run LAMMPS
In this cell, we create the LAMMPS input file BS-Co-Cr-Cu-Fe-Ni.in and a log file log-Co-Cr-Cu-Fe-Ni.lammps. In the input file, we define parameters like the lattice parameter, mass of each atom, and the atom types.

In [ ]:
# Build LAMMPS input file
# assigns lammps input file to variable lammps_name 
lammps_name = 'BS-%s.in' % '-'.join(material)
# assign lammps log file to variable log_name 
log_name = 'log-%s.lammps' % '-'.join(material) 

# assigns information in ''' ''' to variable inputfile
input_file = '''
#LAMMPS input file
    #Initialization: set parameters before before atoms are read-in from a file or created
    boundary        p p p
    units           metal 

    #Atom definition: 
    lattice         {crystal_structure} {lattice_parameter}
    region          box2 block 0 {box_length} 0 {box_length} 0 {box_length}
    create_box      5 box2
    create_atoms    5 box
    mass            1 58.933
    mass            2 51.996
    mass            3 63.546
    mass            4 55.845
    mass            5 58.6934
    
    include         atom_types.txt

    #Settings
    #See LAMMPS documentation of pair_style/pair_coeff for correct inputs
    pair_style      hybrid/overlay eam/alloy zero 12.0 
    pair_coeff      * * eam/alloy {model_name} Co Cr Cu Fe Ni 
    pair_coeff      * * zero
    
    compute bs all sna/atom 1.1 0.99363 8 {cutoff} {cutoff} {cutoff} {cutoff} {cutoff} 27 24 29 26 28 diagonal 3 bzeroflag 0
    dump            ml all custom 1 bs_pre.dump id element c_bs[*]
    dump_modify     ml element Co Cr Cu Fe Ni sort id
    dump            traj all custom 1 xyz.dump id type x y z
    dump_modify     traj element Co Cr Cu Fe Ni sort id
    run 0

'''.format(**inputs) 

# write to file
with open(lammps_name, "w") as f:
    f.write(input_file) 

### LAMMPS execution script
We create and run the LAMMPS execution script (run_lammps.sh). The output you see below the cell is generated from running LAMMPS.

In [ ]:
# Build LAMMPS execution script
write_string = '''
#!/bin/sh
lammpsInput=$1
logname=$2

. /etc/environ.sh

if [ -n "${ANACONDA_CHOICE}" ] ; then
   unuse -e ${ANACONDA_CHOICE}
fi


use -e -r lammps-22Aug18

lmp_serial -in ${lammpsInput} -l ${logname}

'''
# makes file run_lammps.sh and writes to it
with open("run_lammps.sh", "w") as f: 
    f.write(write_string) 
    
# Run lammps simulation
!bash run_lammps.sh {lammps_name} {log_name} 

## 3. Identify Nearest Neighbors
We identify the 12 nearest neighbors of each atom. This code cell creates a script (neighbors.py) and executes it. The output shows the nearest neighbors for each atom using the unique atom identification number.

In [ ]:
py_inputs = {
    'num_atoms': num_atoms,
}

f = open("neighbors.py", "w")

writestring = '''

from ovito.io import import_file
from ovito.data import NearestNeighborFinder
from ovito.io import *
from ovito.data import *
from ovito.modifiers import *
import numpy as np

###############################################################

###############################################################

## Initialize arrays
N = 12

pipeline = import_file("xyz.dump")
data = pipeline.compute()


# Initialize neighbor finder object.
# Visit the 12 nearest neighbors of each particle.
N = 12
finder = NearestNeighborFinder(N, data)

# Prefetch the property array containing the particle type information:

neighbors = np.zeros(({num_atoms},N),int)
neighbor_list = []
# Loop over all input particles:
for index in range({num_atoms}):
    print("Nearest neighbors of particle %i:" % index)
#   # Iterate over the neighbors of the current particle, starting with the closest:
    for neigh in finder.find(index):
        print(neigh.index)
        neighbor_list.append(neigh.index)
    neighbors[index] = neighbor_list
    neighbor_list.clear()
print(np.shape(neighbors))


np.savetxt('neighbors.txt',neighbors)


###############################################################


'''.format(**py_inputs)
f.write(writestring)
f.close()

! /apps/share64/debian7/ovito/ovito-2.9.0/bin/ovitos neighbors.py

### List Atom IDs
We list all atom ids and the type (Cr, Fe, Co, Ni, Cu) in a pandas dataframe. We can use this information to determine the atom type for each atom's 12 nearest neighbors.

In [ ]:
# read nearest neighbor data from file
file = open('xyz.dump','r')
lines = file.readlines()
file.close()

# parse data
del lines[0:9]
ID_list = []
for line in lines:
    split = line.split()
    ID_list.append(split)
ID_list = np.array(ID_list)

# store data in pandas DataFrame, df_IDs
df_IDs = pd.DataFrame(ID_list,columns=['ID','Type','X','Y','Z'])

df_IDs.drop(['X','Y','Z'],axis=1,inplace=True)
display(df_IDs)

### Specify Atom Types of Nearest Neighbors
We determine the atom types of the 12 nearest neighbors of each atom. Then we store this information in a data frame, and save the nearest neighbors data to the file nearest_neighbors.csv.

In [ ]:
neighbors = np.loadtxt('neighbors.txt') + 1
ID_float = ID_list.astype(np.float)
final_type = []
for k in np.arange(0,len(neighbors)):
    type_row = []
    near_list = neighbors[k,:]
    for item in near_list:
        itemindex = (np.where((ID_float[:,0])==item))
        type_row.append(ID_float[int(itemindex[0]),1])
        
    final_type.append(type_row)
    del type_row

df_neighbors = pd.DataFrame(final_type,columns=['N1','N2','N3','N4','N5','N6','N7','N8','N9','N10','N11','N12'])
df_combined = pd.concat([df_IDs,df_neighbors],axis=1)
display(df_combined)
df_combined.to_csv("nearest_neighbors.csv")

### Save Bispectrum Coefficients
We list the bispectrum coefficients for each atom and save them in the file bs_coeffs.json.

In [ ]:
# determine bispectrum coefficients
file = open('bs_pre.dump','r')
lines = file.readlines()

del lines[0:8]
print(lines[0])
header = lines[0].split()
header = header[2:]
print(header)
del lines[0]
BS_list = []
for line in lines:
    split = line.split()
    BS_list.append(split)
BS_list = np.array(BS_list)


df_BS = pd.DataFrame(BS_list,columns=header)
display(df_BS)

# store bispectrum coefficients in a file
elements = df_BS['element'].tolist()

df_BS_only = df_BS.drop(['id', 'element'], axis = 1)

BS_array = df_BS_only.to_numpy()

BS_list = BS_array.tolist()

data = {
    'element': elements,
    'Unrelaxed_Bispectrum_Coefficients': BS_list
}
import json
with open('bs_coeffs.json', 'w') as fp:
    json.dump(data, fp)

In [ ]:
print("Done!")